## Factors
Factor 是一個將資產與時刻轉換為數字的函數
```
F(asset, timestamp) -> float
```
在 Pipeline 中，Factors 是最常被使用的項目，代表輸出一個數值的 `result` 的任何計算的結果。Factors 要求輸入一個資料欄位與窗口長度。

在 Pipeline 中最簡單的 factors 為 __built-in factors__。__Built-in factors__ 是為了執行常用計算預先建立的。作為第一個例子，讓我們創建一個 factor 來計算過去十天的平均收盤價。我們可以使用計算輸入資料（收盤價）平均價值的 __Built-in factor__ `SimpleMovingAverage`（指定窗口長度為10天）。為了實現這個功能，我們需要匯入 __Built-in factor__ `SimpleMovingAverage` factor 以及 TWEquityPricing 資料集。

In [10]:
import os
import pandas as pd
os.environ['TEJAPI_BASE'] = 'https://api.tej.com.tw'
os.environ['TEJAPI_KEY'] = 'your key'
os.environ['ticker'] = '2330 2337'
os.environ['mdate'] = '20070101 20230701'

!zipline ingest -b tquant

from zipline.pipeline import Pipeline
from zipline.master import run_pipeline
from zipline.pipeline.data import TWEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

Merging daily equity files:


[2023-08-08 05:09:11.892037] INFO: zipline.data.bundles.core: Ingesting tquant.


### Creating a Factor
讓我們回到上一堂課的 `make_pipeline` function 接著實例化一個 `SimpleMovingAverage` factor。為了創建一個 `SimpleMovingAverage` factor，我們可以對 `SimpleMovingAverage` 建構子輸入兩個參數：__inputs__（必須為一個 `BoundColumn` 的 `list` 物件），以及 __window_length__（必須為一個表示移動平均線計算應該接收多少天數據的整數）。  
我們會在之後更深度地討論 `BoundColumn`，目前我們只需要知道 `BoundColumn` 是一個表示何種類型的資料應該被傳入 `Factor` 的物件。  
  
下方程式碼將創建一個計算證券10天平均收盤價的 `Factor`。



In [6]:
mean_close_10 = SimpleMovingAverage(inputs=[TWEquityPricing.close], window_length=10)

請注意重要的一點是創建 factor 並不完全地等於執行一個計算。創建一個 factor 就像是定義一個 function。要執行一個計算，我們需要將 Factor 加入 pipeline 並執行它。

### Adding a Factor to a Pipeline
讓我們更新原本的空 pipeline 來讓它計算我們新的移動平均因子。首先，先移動我們的因子實體到 `make_pipeline`。接著，我們可以藉由將讓 `columns` 參數（將列名映射到 factors、 filters 或 classifiers 的字典）輸入 pipeline 去計算 factor 。   
更新後的 `make_pipeline` function 看起來應該像這樣：

In [7]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[TWEquityPricing.close], window_length=10)
    
    return Pipeline(
        columns={
            '10_day_mean_close': mean_close_10
        }
    )

再來，建立pipeline、執行，接著展示結果。

In [8]:
result = run_pipeline(make_pipeline(), '2023-05-05', '2023-05-05')
result

10_day_mean_close
2023-05-05 00:00:00+00:00 Equity(0 [2330])            501.100
                          Equity(1 [2337])             32.935

現在在我們的 pipeline 中有一個有著所有大約80隻證券（未顯示全部）10天平均收盤價的欄位。請注意每一列對應著特定證券在特定儲存日期的計算結果。`DataFrame` 有一個 [MultiIndex](http://pandas.pydata.org/pandas-docs/version/0.16.2/advanced.html) 的功能，它的第一層 __index__ 為執行計算時的日期，而第二層 __index__ 為對應證券的權益物件。舉例來說，第一列（`2023-05-05 00:00:00+00:00`, `Equity(0 [1101])`）會包含在 2023/5/5時，證券代碼為 1101 的 `mean_close_10` factor 的結果。

如果 pipeline 計算天數多於一天，輸出會看起來像這樣：

In [11]:
result = run_pipeline(make_pipeline(), '2018-01-03', '2022-12-30')
result

10_day_mean_close
2018-01-03 00:00:00+00:00 Equity(0 [2330])            226.950
                          Equity(1 [2337])             42.500
2018-01-04 00:00:00+00:00 Equity(0 [2330])            228.150
                          Equity(1 [2337])             42.565
2018-01-05 00:00:00+00:00 Equity(0 [2330])            229.650
...                                                       ...
2022-12-28 00:00:00+00:00 Equity(1 [2337])             34.850
2022-12-29 00:00:00+00:00 Equity(0 [2330])            462.200
                          Equity(1 [2337])             34.680
2022-12-30 00:00:00+00:00 Equity(0 [2330])            458.750
                          Equity(1 [2337])             34.530

[2446 rows x 1 columns]

注意：透過 `Pipeline.add` 方法，factors 也可以被加入至已經存在的 `Pipeline` 實體中。使用 `add` 看起來會像這樣：

```
my_pipe = Pipeline()
f1 = SomeFactor(...)
my_pipe.add(f1, 'f1')
```

### Latest
`Latest` 是最常使用的 `Factor`。`Latest` factor 取得指定欄位中時間距離最近的資料。這個 factor 因為太常使用以至於它實例化的方式與其他 factors 不一樣。取得資料欄位的最近數值的最好方法是呼叫它的 `.latest` 屬性。作為示範，我們更新 `Pipeline` 以創建一個最近 __收盤價__ 的 factor ，然後把它加入 pipeline:


In [12]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[TWEquityPricing.close], window_length=10)
    latest_close = TWEquityPricing.close.latest

    return Pipeline(
        columns={
            '10_day_mean_close': mean_close_10,
            'latest_close_price': latest_close
        }
    )

現在，當我們再次執行 pipeline 時，產出的 DataFrame 會包含兩個欄位。一個欄位記錄每隻證券的10日平均收盤價，另一個記錄最近收盤價。

In [13]:
result = run_pipeline(make_pipeline(), '2023-05-05', '2023-05-05')
result.head(5)

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\zipline\pipeline\engine.py:407: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  root_mask = self._compute_root_mask(


10_day_mean_close  \
2023-05-05 00:00:00+00:00 Equity(0 [2330])            501.100   
                          Equity(1 [2337])             32.935   

                                            latest_close_price  
2023-05-05 00:00:00+00:00 Equity(0 [2330])               498.0  
                          Equity(1 [2337])                32.6

`.latest` 有時可以回傳 `Factors` 以外的東西。我們會在之後的課程示範其他可能回傳的資料類型。

## Default Inputs
有些 factors 有著不可被更改的預設輸入。例如 `VWAP` __built-in factor__ 永遠都是從`TWEquityPricing.close` 和 `TWEquityPricing.volume` 計算而來的。當一個 factor 永遠由同樣一個 `BoundColumns` 計算而來時，我們會說這個建構子沒有指定的 `inputs`。

In [14]:
from zipline.pipeline.factors import VWAP
vwap = VWAP(window_length=10)